In [2]:
import os
import re
import pandas as pd
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import SeqIO

In [2]:

files = os.listdir('out')

for f in files:
    targets = set()

    with open(f'out/{f}', 'r') as inf:
        lines = inf.readlines()

        for l in lines:
            if l.startswith(','):
                continue

            targets.add(l.split(',')[3])

    records = SeqIO.parse(f'../../../ALLEGRO_Fungi_Downloader/data/fourdbs_concat_6_genes/cds/{f}', 'fasta')

    with open(f'target_sequences/{f}', 'w') as outf:
        for record in records:
            if str(record.id) in targets:
                SeqIO.write(record, outf, 'fasta')       

In [6]:
import os
import re
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

# Define the input and output directories
input_dir = 'target_sequences'
output_dir = 'translated_prot_sequences'

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Process each file in the input directory
for filename in os.listdir(input_dir):
    if filename.endswith('.fna'):
        input_path = os.path.join(input_dir, filename)
        output_path = os.path.join(output_dir, filename)

        # Read the nucleotide sequences from the input file
        nucleotide_records = SeqIO.parse(input_path, 'fasta')
        
        # Translate the nucleotide sequences to protein sequences
        protein_records = []
        for record in nucleotide_records:
            new_id = record.id
            s = re.search(r'\[protein_id=(.*?)\]', record.description)
            if s:
                new_id = s.group(1)
                
            translated_seq = record.seq.translate(to_stop=True)
            protein_record = SeqRecord(translated_seq, id=new_id, description=record.description)
            protein_records.append(protein_record)
        
        # Write the translated protein sequences to the output file
        with open(output_path, 'w') as output_file:
            SeqIO.write(protein_records, output_file, 'fasta')

print("Translation completed and saved to 'translated_prot_sequences' folder.")


/home/amohs002/miniconda3/envs/allegro/lib/python3.10/site-packages/Bio/Seq.py:2334: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(


Translation completed and saved to 'translated_prot_sequences' folder.


In [12]:
import re

files = os.listdir('target_sequences')

prot_re = r'\[protein_id=(.*?)\]'

for f in files:
    records = SeqIO.parse(f'target_sequences/{f}', 'fasta')

    collected_prot_ids = list()
    for r in records:
        m = re.search(prot_re, r.description)

        if m:
            collected_prot_ids.append(m.group(1))
    
    prot_f_name = f.replace('_cds.fna', '.faa')
    records = SeqIO.parse(f'../../../ALLEGRO_Fungi_Downloader/data/fourdbs_concat_6_genes/proteomes/{prot_f_name}', 'fasta')
    
    with open(f'target_prot_sequences/{prot_f_name}', 'w') as outf:
        for r in records:
            if str(r.id) in collected_prot_ids:
                SeqIO.write(r, outf, 'fasta')